In [ ]:
!pip install -q pandas pyreadr numpy scipy matplotlib scikit-learn ipykernel jupyter pytest nltk sentence-transformers
import os
import re

import kagglehub

import pandas as pd

import pyreadr

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sentence_transformers import SentenceTransformer

import nltk
from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
result = pyreadr.read_r('scotus_data/scotus_filtered.rda')

In [ ]:
df = result["scotus_filtered"]

Embeddings

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
t5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [ ]:
df["text_embedding"] = df["text"].apply(lambda x: model.encode(x))

In [ ]:
df["t5_text_embedding"] = df["text"].apply(lambda x: t5_model.encode(x))

In [ ]:
pd.DataFrame(df.t5_text_embedding.tolist(), index= df.index).to_csv("scotus/t5_scotus_embeddings.csv", index=False)

In [ ]:
pd.DataFrame(df.text_embedding.tolist(), index= df.index).to_csv("scotus/paraphrase_scotus_embeddings.csv", index=False)

In [22]:
pd.read_csv("t5-scotus_embeddings.csv").drop(columns=["Unnamed: 0"]).to_csv("t5_scotus_embeddings.csv")

Unigram

In [23]:
vectorizer = CountVectorizer(
    max_features=1000,
    stop_words=stopwords.words('english'),
    token_pattern=r"[a-zA-Z]+"
)  # Limit to top 1000 words
word_counts = vectorizer.fit_transform(
    df["text"],
)

unigram_df = pd.DataFrame(
    word_counts.toarray(),
    columns=vectorizer.get_feature_names_out()
)

In [24]:
unigram_df.to_csv("scotus/unigram.csv", index=False)

Bigram

In [25]:
bigram_vectorizer = CountVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    max_features=1000,
    stop_words="english",
    token_pattern=r"[a-zA-Z]+"
)
word_counts = bigram_vectorizer.fit_transform(
    df["text"],
)
bigram_df = pd.DataFrame(
    word_counts.toarray(),
    columns=bigram_vectorizer.get_feature_names_out()
)

In [26]:
bigram_df.to_csv("scotus/bigram.csv", index=False)

Unigram Smoothed

In [27]:
smoothed_unigram = unigram_df + 1

In [28]:
smoothed_unigram.to_csv("scotus/smoothed_unigram.csv")

TFIDF

In [30]:

tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,
    stop_words=stopwords.words('english'),
    token_pattern=r"[a-zA-Z]+"
)
word_counts = tfidf_vectorizer.fit_transform(
    df["text"]
)

tfidf_df = pd.DataFrame(
    word_counts.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out()
)

In [31]:
tfidf_df.to_csv("scotus/tfidf.csv",index=False)

SBERT

In [ ]:
from sentence_transformers import SentenceTransformer

sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
df["sbert_total_text_embedding"] = df["text"].apply(lambda x: sbert_model.encode(x))

In [ ]:
sbert_embedding_df = pd.DataFrame(df.sbert_total_text_embedding.tolist(), index= df.index)
sbert_embedding_df.to_csv("linkedin/sbert.csv")

Doc2Vec

In [ ]:
!pip install gensim

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_data = [TaggedDocument(words=row.split(), tags=[str(idx)]) for idx, row in enumerate(df["text"])]

d2v_model = Doc2Vec(
    tagged_data,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    epochs=50
)

In [ ]:
df["d2v_total_text_embedding"] = df["total_text"].apply(lambda x: d2v_model.infer_vector(x.split()))

In [ ]:
d2v_embedding_df = pd.DataFrame(df.d2v_total_text_embedding.tolist(), index= df.index)
d2v_embedding_df.to_csv("linkedin/doc2vec.csv")